In [1]:
import numpy as np

from skimage.io import imshow
from skimage.transform import resize

from sklearn.model_selection import train_test_split

from collections import namedtuple

from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Activation
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
%matplotlib inline

Using TensorFlow backend.


In [2]:
def get_train_data(path='public_data/00_input/train/'):
    from skimage.io import imread
    with open(path + 'gt.csv') as navigation_file:
        splits = [line.split(',') for line in navigation_file]
        points_number = int((len(splits[0]) - 1) / 2)
        train_x = np.array([imread(path + 'images/' + split[0]) for split in splits[1:]])
        train_y = np.array([[int(x) for x in split[1:]] for split in splits[1:]])
        return train_x, train_y, points_number

In [3]:
def get_colored_data(x, y):
    return (
        np.array([image for image in x if len(np.shape(image)) == 3]),
        np.array([answer for image, answer in zip(x, y) if len(np.shape(image)) == 3])
    )

In [4]:
all_x, all_y, points_number = get_train_data()

In [5]:
def get_resized_train_data(train_x, train_y, shape=(64, 64, 3)):
    resized_x = np.array([resize(image, shape, mode='reflect') for image in train_x])
    resized_y = np.array([[
            int(coordinate * shape[i % 2] / np.shape(image)[i % 2])
            for i, coordinate in enumerate(answer)
        ] for image, answer in zip(train_x, train_y)
    ])
    return resized_x, resized_y

In [6]:
colored_x, colored_y = get_colored_data(all_x, all_y)

In [7]:
resized_x, resized_y = get_resized_train_data(colored_x, colored_y)
input_shape=np.shape(resized_x[0])

In [8]:
train_x, test_x, train_y, test_y = train_test_split(
    resized_x, resized_y, test_size=0.3, random_state=42
)

In [9]:
class Augmentation:
    def __init__(self,
                 featurewise_center=True,
                 featurewise_std_normalization=True,
                 rotation=20,
                 width_shift=0.2,
                 height_shift=0.2,
                 horizontal_flip=True):
        self.featurewise_center = featurewise_cente
        self.featurewise_std_normalization = featurewise_std_normalization
        self.rotation = rotation
        self.width_shift = width_shift
        self.height_shift = height_shift
        self.horizontal_flip = horizontal_flip

In [10]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
)

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
)

In [11]:
datagen.fit(train_x)

In [12]:
regularization_lambda = 0#0.0001

def add_Conv2D_relu_BatchNorm(model, n_filter, filters_size, input_shape=None):
    if input_shape is None:
        model.add(Conv2D(
            n_filter, filters_size, padding='same',
            kernel_regularizer=regularizers.l2(regularization_lambda)
        ))
    else:
        model.add(Conv2D(
            n_filter, filters_size, padding='same',
            kernel_regularizer=regularizers.l2(regularization_lambda),
            input_shape=input_shape
        ))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

model = Sequential()

add_Conv2D_relu_BatchNorm(model, 64, (3, 3), input_shape)
add_Conv2D_relu_BatchNorm(model, 64, (3, 3))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))
model.add(Dropout(0.25))

add_Conv2D_relu_BatchNorm(model, 128, (3, 3))
add_Conv2D_relu_BatchNorm(model, 128, (3, 3))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))
model.add(Dropout(0.25))

add_Conv2D_relu_BatchNorm(model, 256, (3, 3))
add_Conv2D_relu_BatchNorm(model, 256, (3, 3))
add_Conv2D_relu_BatchNorm(model, 256, (3, 3))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))
model.add(Dropout(0.25))

add_Conv2D_relu_BatchNorm(model, 512, (3, 3))
add_Conv2D_relu_BatchNorm(model, 512, (3, 3))
add_Conv2D_relu_BatchNorm(model, 512, (3, 3))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten(name='flatten'))
model.add(Dense(4096, activation='relu', kernel_regularizer=regularizers.l2(regularization_lambda)))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu', kernel_regularizer=regularizers.l2(regularization_lambda)))

model.add(Dense(points_number * 2, activation='relu'))

In [ ]:
model.compile(loss='mean_squared_error',
              optimizer=Adam(),
              metrics=['mse'])

In [ ]:
model.fit_generator(datagen.flow(train_x, train_y, batch_size=32),
                    steps_per_epoch=int(len(train_y) / 32), epochs=1000)

Epoch 1/1000
130/130 [==============================] - 50s - loss: 167.1856 - mean_squared_error: 167.1856    
Epoch 2/1000
130/130 [==============================] - 38s - loss: 117.6135 - mean_squared_error: 117.6135    
Epoch 3/1000
130/130 [==============================] - 38s - loss: 115.0178 - mean_squared_error: 115.0178    
Epoch 4/1000
130/130 [==============================] - 38s - loss: 115.2481 - mean_squared_error: 115.2481    
Epoch 5/1000
130/130 [==============================] - 38s - loss: 113.8043 - mean_squared_error: 113.8043    
Epoch 6/1000
130/130 [==============================] - 38s - loss: 112.4664 - mean_squared_error: 112.4664    
Epoch 7/1000
130/130 [==============================] - 38s - loss: 112.7495 - mean_squared_error: 112.7495    
Epoch 8/1000
130/130 [==============================] - 38s - loss: 109.8013 - mean_squared_error: 109.8013    
Epoch 9/1000
130/130 [==============================] - 38s - loss: 109.2957 - mean_squared_error: 109.2

130/130 [==============================] - 38s - loss: 81.7034 - mean_squared_error: 81.7034    
Epoch 75/1000
130/130 [==============================] - 38s - loss: 81.8155 - mean_squared_error: 81.8155    
Epoch 76/1000
130/130 [==============================] - 38s - loss: 81.6597 - mean_squared_error: 81.6597    
Epoch 77/1000
130/130 [==============================] - 38s - loss: 81.7650 - mean_squared_error: 81.7650    
Epoch 78/1000
130/130 [==============================] - 38s - loss: 81.1026 - mean_squared_error: 81.1026    
Epoch 79/1000
130/130 [==============================] - 38s - loss: 81.6469 - mean_squared_error: 81.6469    
Epoch 80/1000
130/130 [==============================] - 38s - loss: 82.1053 - mean_squared_error: 82.1053    
Epoch 81/1000
130/130 [==============================] - 38s - loss: 81.1090 - mean_squared_error: 81.1090    
Epoch 82/1000
130/130 [==============================] - 38s - loss: 81.8424 - mean_squared_error: 81.8424    
Epoch 83/1000
1

130/130 [==============================] - 38s - loss: 81.1568 - mean_squared_error: 81.1568    
Epoch 148/1000
130/130 [==============================] - 38s - loss: 81.4053 - mean_squared_error: 81.4053    
Epoch 149/1000
130/130 [==============================] - 38s - loss: 81.2090 - mean_squared_error: 81.2090    
Epoch 150/1000
130/130 [==============================] - 38s - loss: 81.2118 - mean_squared_error: 81.2118    
Epoch 151/1000
130/130 [==============================] - 38s - loss: 81.6796 - mean_squared_error: 81.6796    
Epoch 152/1000
130/130 [==============================] - 38s - loss: 80.7742 - mean_squared_error: 80.7742    
Epoch 153/1000
130/130 [==============================] - 38s - loss: 81.3808 - mean_squared_error: 81.3808    
Epoch 154/1000
130/130 [==============================] - 38s - loss: 81.2606 - mean_squared_error: 81.2606    
Epoch 155/1000
130/130 [==============================] - 38s - loss: 81.4174 - mean_squared_error: 81.4174    
Epoch 1

130/130 [==============================] - 38s - loss: 12.4661 - mean_squared_error: 12.4661    
Epoch 221/1000
130/130 [==============================] - 38s - loss: 12.6439 - mean_squared_error: 12.6439    
Epoch 222/1000
130/130 [==============================] - 38s - loss: 12.6303 - mean_squared_error: 12.6303    
Epoch 223/1000
130/130 [==============================] - 38s - loss: 12.8063 - mean_squared_error: 12.8063    
Epoch 224/1000
130/130 [==============================] - 38s - loss: 12.2810 - mean_squared_error: 12.2810    
Epoch 225/1000
130/130 [==============================] - 38s - loss: 12.5790 - mean_squared_error: 12.5790    
Epoch 226/1000
130/130 [==============================] - 38s - loss: 12.6129 - mean_squared_error: 12.6129    
Epoch 227/1000
130/130 [==============================] - 38s - loss: 12.9218 - mean_squared_error: 12.9218    
Epoch 228/1000
130/130 [==============================] - 38s - loss: 12.6056 - mean_squared_error: 12.6056    
Epoch 2

130/130 [==============================] - 38s - loss: 13.1174 - mean_squared_error: 13.1174    
Epoch 294/1000
130/130 [==============================] - 38s - loss: 12.1873 - mean_squared_error: 12.1873    
Epoch 295/1000
130/130 [==============================] - 38s - loss: 12.8685 - mean_squared_error: 12.8685    
Epoch 296/1000
130/130 [==============================] - 38s - loss: 12.4103 - mean_squared_error: 12.4103    
Epoch 297/1000
130/130 [==============================] - 38s - loss: 12.6542 - mean_squared_error: 12.6542    
Epoch 298/1000
130/130 [==============================] - 38s - loss: 12.0194 - mean_squared_error: 12.0194    
Epoch 299/1000
130/130 [==============================] - 38s - loss: 12.5241 - mean_squared_error: 12.5241    
Epoch 300/1000
130/130 [==============================] - 38s - loss: 12.3729 - mean_squared_error: 12.3729    
Epoch 301/1000
130/130 [==============================] - 38s - loss: 12.4345 - mean_squared_error: 12.4345    
Epoch 3

130/130 [==============================] - 38s - loss: 12.1586 - mean_squared_error: 12.1586    
Epoch 367/1000
130/130 [==============================] - 38s - loss: 11.9457 - mean_squared_error: 11.9457    
Epoch 368/1000
130/130 [==============================] - 38s - loss: 12.2160 - mean_squared_error: 12.2160    
Epoch 369/1000
130/130 [==============================] - 38s - loss: 12.2832 - mean_squared_error: 12.2832    
Epoch 370/1000
130/130 [==============================] - 38s - loss: 11.9648 - mean_squared_error: 11.9648    
Epoch 371/1000
130/130 [==============================] - 38s - loss: 12.3032 - mean_squared_error: 12.3032    
Epoch 372/1000
130/130 [==============================] - 38s - loss: 12.0654 - mean_squared_error: 12.0654    
Epoch 373/1000
130/130 [==============================] - 38s - loss: 11.8018 - mean_squared_error: 11.8018    
Epoch 374/1000
130/130 [==============================] - 38s - loss: 12.6866 - mean_squared_error: 12.6866    
Epoch 3

130/130 [==============================] - 38s - loss: 11.6934 - mean_squared_error: 11.6934    
Epoch 440/1000
130/130 [==============================] - 38s - loss: 11.8603 - mean_squared_error: 11.8603    
Epoch 441/1000
130/130 [==============================] - 38s - loss: 11.4982 - mean_squared_error: 11.4982    
Epoch 442/1000
130/130 [==============================] - 38s - loss: 11.8999 - mean_squared_error: 11.8999    
Epoch 443/1000
130/130 [==============================] - 38s - loss: 11.6530 - mean_squared_error: 11.6530    
Epoch 444/1000
130/130 [==============================] - 38s - loss: 11.9760 - mean_squared_error: 11.9760    
Epoch 445/1000
130/130 [==============================] - 38s - loss: 11.9189 - mean_squared_error: 11.9189    
Epoch 446/1000
130/130 [==============================] - 38s - loss: 11.7897 - mean_squared_error: 11.7897    
Epoch 447/1000
130/130 [==============================] - 38s - loss: 11.6283 - mean_squared_error: 11.6283    
Epoch 4

130/130 [==============================] - 38s - loss: 10.2504 - mean_squared_error: 10.2504    
Epoch 513/1000
130/130 [==============================] - 38s - loss: 10.1229 - mean_squared_error: 10.1229    
Epoch 514/1000
130/130 [==============================] - 38s - loss: 10.2236 - mean_squared_error: 10.2236    
Epoch 515/1000
130/130 [==============================] - 38s - loss: 10.7366 - mean_squared_error: 10.7366    
Epoch 516/1000
130/130 [==============================] - 38s - loss: 10.1365 - mean_squared_error: 10.1365    
Epoch 517/1000
130/130 [==============================] - 38s - loss: 10.4256 - mean_squared_error: 10.4256    
Epoch 518/1000
130/130 [==============================] - 38s - loss: 9.4887 - mean_squared_error: 9.4887    
Epoch 519/1000
130/130 [==============================] - 38s - loss: 10.3650 - mean_squared_error: 10.3650    
Epoch 520/1000
130/130 [==============================] - 38s - loss: 10.0674 - mean_squared_error: 10.0674    
Epoch 521